In [14]:
import os
import sys
import csv
from file_utils import *
import re

In [15]:
list_files = []

for x in range(5):
    sess_name = "Session" + str(x+1)
    print(sess_name)
    path = "/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/" + sess_name + "/sentences/wav/"
    print(path)
    #file_search(path, list_files)
    #list_files = sorted(list_files)

    # print sess_name + ", #sum files: " + str(len(list_files))

Session1
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session1/sentences/wav/
Session2
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session2/sentences/wav/
Session3
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session3/sentences/wav/
Session4
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session4/sentences/wav/
Session5
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session5/sentences/wav/


In [16]:
useful_regex = re.compile(r'\[.+\]\n', re.IGNORECASE)


In [19]:
file_path = '/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro01.txt'

In [20]:
with open(file_path) as f:
    file_content = f.read()
    
info_lines = re.findall(useful_regex, file_content)

In [21]:
for l in info_lines[1:10]:
    print(l.strip().split('\t'))

['[6.2901 - 8.2357]', 'Ses01F_impro01_F000', 'neu', '[2.5000, 2.5000, 2.5000]']
['[10.0100 - 11.3925]', 'Ses01F_impro01_F001', 'neu', '[2.5000, 2.5000, 2.5000]']
['[14.8872 - 18.0175]', 'Ses01F_impro01_F002', 'neu', '[2.5000, 2.5000, 2.5000]']
['[19.2900 - 20.7875]', 'Ses01F_impro01_F003', 'xxx', '[2.5000, 3.0000, 3.0000]']
['[21.3257 - 24.7400]', 'Ses01F_impro01_F004', 'xxx', '[2.5000, 3.0000, 2.5000]']
['[27.4600 - 31.4900]', 'Ses01F_impro01_F005', 'neu', '[2.5000, 3.5000, 2.0000]']
['[38.9650 - 43.5900]', 'Ses01F_impro01_F006', 'fru', '[2.0000, 3.5000, 3.5000]']
['[46.5800 - 52.1900]', 'Ses01F_impro01_F007', 'fru', '[2.5000, 3.5000, 3.5000]']
['[56.1600 - 58.8225]', 'Ses01F_impro01_F008', 'fru', '[2.0000, 3.5000, 3.5000]']


In [47]:
info_line = re.compile(r'\[.+\]\n', re.IGNORECASE)

start_times, end_times, wav_file_names, emotions, vals, acts, doms = [], [], [], [], [], [], []
import pdb
for sess in range(1, 5):
    emo_evaluation_dir = '/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session{}/dialog/EmoEvaluation/'.format(sess)
    evaluation_files = [l for l in os.listdir(emo_evaluation_dir) if 'Ses' in l]
    for file in evaluation_files:
        #
        #
        x = re.search("^Ses.*", file)
        #print(x, file)
        #pdb.set_trace()
        if x == None:
            continue
        with open(emo_evaluation_dir + file) as f:
            content = f.read()
        info_lines = re.findall(info_line, content)
        for line in info_lines[1:]:  # the first line is a header
            #pdb.set_trace()   
            start_end_time, wav_file_name, emotion, val_act_dom = line.strip().split('\t')
            start_time, end_time = start_end_time[1:-1].split('-')
            val, act, dom = val_act_dom[1:-1].split(',')
            val, act, dom = float(val), float(act), float(dom)
            start_time, end_time = float(start_time), float(end_time)
            start_times.append(start_time)
            end_times.append(end_time)
            wav_file_names.append(wav_file_name)
            emotions.append(emotion)
            vals.append(val)
            acts.append(act)
            doms.append(dom)

In [48]:
import pandas as pd

df_iemocap = pd.DataFrame(columns=['start_time', 'end_time', 'wav_file', 'emotion', 'val', 'act', 'dom'])

df_iemocap['start_time'] = start_times
df_iemocap['end_time'] = end_times
df_iemocap['wav_file'] = wav_file_names
df_iemocap['emotion'] = emotions
df_iemocap['val'] = vals
df_iemocap['act'] = acts
df_iemocap['dom'] = doms

df_iemocap.tail()

,start_time,end_time,wav_file,emotion,val,act,dom
7864,139.08,141.80,Ses04F_impro01_M021,ang,2.0,4.0,4.5
7865,145.26,148.57,Ses04F_impro01_M022,fru,1.5,4.0,4.5
7866,148.69,151.21,Ses04F_impro01_M023,ang,1.5,4.5,5.0
7867,151.22,155.47,Ses04F_impro01_M024,ang,1.0,4.5,5.0
7868,165.43,168.51,Ses04F_impro01_M025,xxx,2.0,3.0,3.5
